In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import os
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/Finnhub')

In [3]:
# !pip install keras

In [4]:
import pandas as pd
import numpy as np
import random
import time
import pickle
import json
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from Statistics import Statistics

import tensorflow as tf
from keras.layers import Dense, LSTM, BatchNormalization, Input, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
from keras.models import Model, Sequential, load_model
from keras import optimizers
import warnings
warnings.filterwarnings("ignore")

import os
SEED = 9
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [5]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [6]:
def extract_names_from_json(json_file_path):
    with open(json_file_path, "r") as f:
        json_data = json.load(f)

    names = []
    for entry in json_data:
        names.append(entry["Symbol"])

    return names

def makeLSTM():
    inputs = Input(shape=(240,3))
    x = LSTM(25,return_sequences=False)(inputs)
    x = Dropout(0.1)(x)
    outputs = Dense(2,activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(),
                          metrics=['accuracy'])
    model.summary()
    return model


def callbacks_req(model_type='LSTM'):
    csv_logger = CSVLogger(model_folder+'/training-log-'+model_type+'-'+str(test_year)+'.csv')
    filepath = model_folder+"/model-" + model_type + '-' + str(test_year) + "-E{epoch:02d}.h5"
    model_checkpoint = ModelCheckpoint(filepath, monitor='val_loss',save_best_only=False, period=1)
    earlyStopping = EarlyStopping(monitor='val_loss',mode='min',patience=10,restore_best_weights=True)
    return [csv_logger,earlyStopping,model_checkpoint]

def reshaper(arr):
    arr = np.array(np.split(arr,3,axis=1))
    arr = np.swapaxes(arr,0,1)
    arr = np.swapaxes(arr,1,2)
    return arr

def trainer(train_data,test_data):
    np.random.shuffle(train_data)
    train_x,train_y,train_ret = train_data[:,2:-2],train_data[:,-1],train_data[:,-2]
    train_x = train_x.astype('float32')
    train_y = train_y.astype('float32')
    train_x = reshaper(train_x)
    train_y = np.reshape(train_y,(-1, 1))
    train_ret = np.reshape(train_ret,(-1, 1))
    enc = OneHotEncoder(handle_unknown='ignore')
    enc.fit(train_y)
    enc_y = enc.transform(train_y).toarray()
    train_ret = np.hstack((np.zeros((len(train_data),1)),train_ret))

    model = makeLSTM()
    callbacks = callbacks_req()

    model.fit(train_x,
              enc_y,
              epochs=1000,
              validation_split=0.2,
              callbacks=callbacks,
              batch_size=512
              )

    dates = list(set(test_data[:,0]))
    predictions = {}
    for day in dates:
        test_d = test_data[test_data[:,0]==day]
        test_d = reshaper(test_d[:,2:-2]).astype('float32')
        predictions[day] = model.predict(test_d)[:,1]
    return model,predictions

def trained(filename,train_data,test_data):
    model = load_model(filename)

    dates = list(set(test_data[:,0]))
    predictions = {}
    for day in dates:
        test_d = test_data[test_data[:,0]==day]
        test_d = np.reshape(test_d[:,2:-2],(len(test_d),240,1))
        predictions[day] = model.predict(test_d)[:,1]
    return model,predictions

def simulate(test_data,predictions):
    rets = pd.DataFrame([],columns=['Long','Short'])
    k = 10
    for day in sorted(predictions.keys()):
        preds = predictions[day]
        test_returns = test_data[test_data[:,0]==day][:,-2]
        top_preds = predictions[day].argsort()[-k:][::-1]
        trans_long = test_returns[top_preds]
        worst_preds = predictions[day].argsort()[:k][::-1]
        trans_short = -test_returns[worst_preds]
        rets.loc[day] = [np.mean(trans_long),np.mean(trans_short)]
    print('Result : ',rets.mean())
    return rets


def create_label(df_open,df_close,perc=[0.5,0.5]):
    if not np.all(df_close.iloc[:,0]==df_open.iloc[:,0]):
        print('Date Index issue')
        return
    perc = [0.]+list(np.cumsum(perc))
    label = (df_close.iloc[:,1:]/df_open.iloc[:,1:]-1).apply(
            lambda x: pd.qcut(x.rank(method='first'),perc,labels=False), axis=1)
    return label[1:]

def create_stock_data(df_close,df_open,st,label,test_year,m=240):
    st_data = pd.DataFrame([])
    st_data['Date'] = list(df_close['Date'])
    st_data['Name'] = [st]*len(st_data)
    daily_change = df_close[st]/df_open[st]-1
    for k in range(m)[::-1]:
        st_data['IntraR'+str(k)] = daily_change.shift(k)
    nextday_ret = (np.array(df_open[st][1:])/np.array(df_close[st][:-1])-1)
    nextday_ret = pd.Series(list(nextday_ret)+[np.nan])
    for k in range(m)[::-1]:
        st_data['NextR'+str(k)] = nextday_ret.shift(k)
    close_change = df_close[st].pct_change()
    for k in range(m)[::-1]:
        st_data['CloseR'+str(k)] = close_change.shift(k)
    st_data['IntraR-future'] = daily_change.shift(-1)
    st_data['label'] = list(label[st])+[np.nan]
    st_data['Month'] = list(df_close['Date'].str[:-3])
    st_data = st_data.dropna()

    trade_year = st_data['Month'].str[:4]
    st_data = st_data.drop(columns=['Month'])
    st_train_data = st_data[trade_year<str(test_year)]
    st_test_data = st_data[trade_year==str(test_year)]
    return np.array(st_train_data),np.array(st_test_data)

def scalar_normalize(train_data,test_data):
    scaler = RobustScaler()
    scaler.fit(train_data[:,2:-2])
    train_data[:,2:-2] = scaler.transform(train_data[:,2:-2])
    test_data[:,2:-2] = scaler.transform(test_data[:,2:-2])

def calculate_accuracy(predictions, test_data):
    accuracies = {}

    for date, daily_predictions in predictions.items():
        filtered_data = test_data[test_data[:, 0] == date]

        # Convert predictions to binary labels
        binary_predictions = [1 if prediction >= 0.5 else 0 for prediction in daily_predictions]

        # Calculate accuracy for the day
        correct_predictions = 0
        for prediction, label in zip(binary_predictions, filtered_data[:, -1]):
            if prediction == label:
                correct_predictions += 1

        accuracy = correct_predictions / len(binary_predictions)
        accuracies[date] = accuracy

    accuracy = np.mean(list(accuracies.values()))

    return accuracy

model_folder = 'models-Intraday-240-3-LSTM'
result_folder = 'results-Intraday-240-3-LSTM'
for directory in [model_folder,result_folder]:
    if not os.path.exists(directory):
        os.makedirs(directory)


In [7]:
json_file_path = "s&p500.json"
names = extract_names_from_json(json_file_path)

result_folder = 'results-Intraday-240-3-LSTM'
for directory in [result_folder]:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [8]:
filename = '/content/gdrive/MyDrive/Colab Notebooks/Finnhub/Open-1996.xlsx'
df_open = pd.read_excel(filename)
filename = '/content/gdrive/MyDrive/Colab Notebooks/Finnhub/Close-1996.xlsx'
df_close = pd.read_excel(filename)

# print(df_open.drop(df_open.columns[df_open.isnull().any()], axis=1, inplace=True))
label = create_label(df_open,df_close)

In [9]:
constituents = {}

for index, row in df_open.iterrows():
  date = row['Date']
  stocks = list(row.dropna().index.drop('Date'))
  constituents[date] = stocks

In [10]:
for test_year in range(2018,2023):

  print('-'*40)
  print(test_year)
  print('-'*40)

  df_close_cur = df_close[(df_close['Date'].str[:4].astype(int) >= test_year -3) & (df_close['Date'].str[:4].astype(int) <= test_year)].reset_index(drop=True)
  df_open_cur = df_open[(df_open['Date'].str[:4].astype(int) >= test_year - 3) & (df_open['Date'].str[:4].astype(int) <= test_year)].reset_index(drop=True)

  # print(df_open_cur.head())
  label = create_label(df_open_cur,df_close_cur)

  earliest_date = ''

  for key in constituents:
      if key.startswith(str(test_year-3)):
          if earliest_date == '':
            earliest_date = key
          elif key < earliest_date:
            earliest_date = key

  stock_names = sorted(list(constituents[earliest_date]))

  start = time.time()
  train_data,test_data = [],[]

  for st in stock_names:
    st_train_data,st_test_data = create_stock_data(df_close_cur,df_open_cur,st,label,test_year)
    train_data.append(st_train_data)
    test_data.append(st_test_data)

  train_data = np.concatenate([x for x in train_data])
  test_data = np.concatenate([x for x in test_data])
  scalar_normalize(train_data,test_data)
  print(train_data.shape,test_data.shape,time.time()-start)

  model,predictions = trainer(train_data,test_data)

  accuracy = calculate_accuracy(predictions, test_data)

  returns = simulate(test_data,predictions)
  returns.to_csv(result_folder+'/avg_daily_rets-'+str(test_year)+'.csv')

  result = Statistics(returns.sum(axis=1))
  print('\nAverage returns prior to transaction charges')
  report = result.report()
  # with open(result_folder+"/avg_returns-" + str(test_year) + ".txt", "a") as myfile:
  #     res = '-'*30 + '\n'
  #     res += str(test_year) + '\n'
  #     res += 'Mean = ' + str(result.mean()) + '\n'
  #     res += 'Sharpe = '+str(result.sharpe()) + '\n'
  #     res += 'Accuracy = ' + str(accuracy) + '\n'
  #     res += '-'*30 + '\n'
  #     myfile.write(res)
  with open(result_folder+"/report-" + str(test_year) + ".txt", "a") as f:
    res = '-'*30 + '\n'
    res += str(test_year) + '\n'
    res += 'Mean = ' + str(result.mean()) + '\n'
    res += 'Standard dev = '+str(result.std()) + '\n'
    res += 'Sharpe ratio = ' + str(result.sharpe()) + '\n'
    res += 'Standard Error = '+str(result.stderr()) + '\n'
    res += 'Share>0 = ' + str(result.pos_perc()) + '\n'
    res += 'Skewness = '+str(result.skewness()) + '\n'
    res += 'Kurtosis = ' + str(result.kurtosis()) + '\n'
    res += 'VaR_1 = '+str(result.VaR(1)) + '\n'
    res += 'VaR_2 = ' + str(result.VaR(2)) + '\n'
    res += 'VaR_5 = '+str(result.VaR(5)) + '\n'
    res += 'CVaR_1 = ' + str(result.CVaR(1)) + '\n'
    res += 'CVaR_2 = '+str(result.CVaR(2)) + '\n'
    res += 'CVaR_5 = '+str(result.CVaR(5)) + '\n'
    res += 'MDD = '+str(result.MDD()) + '\n'
    res += 'Percentiles = '+str(result.percentiles()) + '\n'
    res += '-'*30 + '\n'
    f.write(res)

----------------------------------------
2018
----------------------------------------
(247036, 724) (120250, 724) 208.8796103000641
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 240, 3)]          0         
                                                                 
 lstm (LSTM)                 (None, 25)                2900      
                                                                 
 dropout (Dropout)           (None, 25)                0         
                                                                 
 dense (Dense)               (None, 2)                 52        
                                                                 
Total params: 2952 (11.53 KB)
Trainable params: 2952 (11.53 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/1000
386/386 [==============================] - 14s 18ms/step - loss: 0.6930 - accuracy: 0.5125 - val_loss: 0.6926 - val_accuracy: 0.5173
Epoch 2/1000
386/386 [==============================] - 5s 14ms/step - loss: 0.6926 - accuracy: 0.5179 - val_loss: 0.6926 - val_accuracy: 0.5174
Epoch 3/1000
386/386 [==============================] - 5s 14ms/step - loss: 0.6924 - accuracy: 0.5185 - val_loss: 0.6927 - val_accuracy: 0.5151
Epoch 4/1000
386/386 [==============================] - 5s 13ms/step - loss: 0.6924 - accuracy: 0.5184 - val_loss: 0.6926 - val_accuracy: 0.5165
Epoch 5/1000
386/386 [==============================] - 5s 14ms/step - loss: 0.6923 - accuracy: 0.5176 - val_loss: 0.6925 - val_accuracy: 0.5186
Epoch 6/1000
386/386 [==============================] - 5s 13ms/step - loss: 0.6922 - accuracy: 0.5192 - val_loss: 0.6929 - val_accuracy: 0.5153
Epoch 7/1000
386/386 [==============================] - 5s 13ms/step - loss: 0.6922 - accuracy: 0.5182 - val_loss: 0.6925 - val_a

Epoch 1/1000
391/391 [==============================] - 8s 18ms/step - loss: 0.6931 - accuracy: 0.5087 - val_loss: 0.6928 - val_accuracy: 0.5073
Epoch 2/1000
391/391 [==============================] - 6s 16ms/step - loss: 0.6927 - accuracy: 0.5112 - val_loss: 0.6925 - val_accuracy: 0.5150
Epoch 3/1000
391/391 [==============================] - 6s 14ms/step - loss: 0.6925 - accuracy: 0.5124 - val_loss: 0.6924 - val_accuracy: 0.5164
Epoch 4/1000
391/391 [==============================] - 5s 14ms/step - loss: 0.6924 - accuracy: 0.5142 - val_loss: 0.6923 - val_accuracy: 0.5160
Epoch 5/1000
391/391 [==============================] - 5s 14ms/step - loss: 0.6924 - accuracy: 0.5133 - val_loss: 0.6926 - val_accuracy: 0.5108
Epoch 6/1000
391/391 [==============================] - 5s 13ms/step - loss: 0.6922 - accuracy: 0.5157 - val_loss: 0.6924 - val_accuracy: 0.5134
Epoch 7/1000
391/391 [==============================] - 5s 13ms/step - loss: 0.6922 - accuracy: 0.5156 - val_loss: 0.6923 - val_ac

Epoch 1/1000
394/394 [==============================] - 9s 18ms/step - loss: 0.6935 - accuracy: 0.5025 - val_loss: 0.6930 - val_accuracy: 0.5044
Epoch 2/1000
394/394 [==============================] - 5s 13ms/step - loss: 0.6931 - accuracy: 0.5062 - val_loss: 0.6931 - val_accuracy: 0.5049
Epoch 3/1000
394/394 [==============================] - 5s 13ms/step - loss: 0.6930 - accuracy: 0.5042 - val_loss: 0.6931 - val_accuracy: 0.5046
Epoch 4/1000
394/394 [==============================] - 5s 13ms/step - loss: 0.6930 - accuracy: 0.5072 - val_loss: 0.6929 - val_accuracy: 0.5064
Epoch 5/1000
394/394 [==============================] - 5s 13ms/step - loss: 0.6929 - accuracy: 0.5083 - val_loss: 0.6931 - val_accuracy: 0.5061
Epoch 6/1000
394/394 [==============================] - 5s 14ms/step - loss: 0.6929 - accuracy: 0.5086 - val_loss: 0.6929 - val_accuracy: 0.5067
Epoch 7/1000
394/394 [==============================] - 5s 14ms/step - loss: 0.6928 - accuracy: 0.5088 - val_loss: 0.6929 - val_ac

Epoch 1/1000
397/397 [==============================] - 8s 17ms/step - loss: 0.6938 - accuracy: 0.5047 - val_loss: 0.6924 - val_accuracy: 0.5110
Epoch 2/1000
397/397 [==============================] - 5s 13ms/step - loss: 0.6927 - accuracy: 0.5110 - val_loss: 0.6922 - val_accuracy: 0.5125
Epoch 3/1000
397/397 [==============================] - 5s 14ms/step - loss: 0.6923 - accuracy: 0.5107 - val_loss: 0.6920 - val_accuracy: 0.5151
Epoch 4/1000
397/397 [==============================] - 5s 14ms/step - loss: 0.6922 - accuracy: 0.5119 - val_loss: 0.6917 - val_accuracy: 0.5154
Epoch 5/1000
397/397 [==============================] - 5s 14ms/step - loss: 0.6919 - accuracy: 0.5131 - val_loss: 0.6915 - val_accuracy: 0.5184
Epoch 6/1000
397/397 [==============================] - 6s 14ms/step - loss: 0.6918 - accuracy: 0.5142 - val_loss: 0.6917 - val_accuracy: 0.5130
Epoch 7/1000
397/397 [==============================] - 5s 13ms/step - loss: 0.6916 - accuracy: 0.5149 - val_loss: 0.6915 - val_ac

Epoch 1/1000
400/400 [==============================] - 9s 18ms/step - loss: 0.6931 - accuracy: 0.5084 - val_loss: 0.6922 - val_accuracy: 0.5156
Epoch 2/1000
400/400 [==============================] - 6s 15ms/step - loss: 0.6926 - accuracy: 0.5102 - val_loss: 0.6920 - val_accuracy: 0.5171
Epoch 3/1000
400/400 [==============================] - 6s 15ms/step - loss: 0.6923 - accuracy: 0.5135 - val_loss: 0.6921 - val_accuracy: 0.5154
Epoch 4/1000
400/400 [==============================] - 5s 13ms/step - loss: 0.6922 - accuracy: 0.5137 - val_loss: 0.6917 - val_accuracy: 0.5180
Epoch 5/1000
400/400 [==============================] - 6s 14ms/step - loss: 0.6920 - accuracy: 0.5150 - val_loss: 0.6916 - val_accuracy: 0.5180
Epoch 6/1000
400/400 [==============================] - 5s 14ms/step - loss: 0.6920 - accuracy: 0.5148 - val_loss: 0.6915 - val_accuracy: 0.5197
Epoch 7/1000
400/400 [==============================] - 6s 15ms/step - loss: 0.6918 - accuracy: 0.5162 - val_loss: 0.6915 - val_ac